In [13]:
import cv2
import mediapipe as mp
import pygame
import time
import math
import numpy as np

# =============================================================================
# Variables Globales y Configuración Inicial
# =============================================================================
selected_points = []     # 4 puntos para transformación en perspectiva
static_keys = False      # Modo: teclas fijas (estáticas)
frontal_view = False     # Vista frontal (transformada)
rotate_180 = False       # Rota el frame 180°
mirror_effect = False    # Efecto espejo

# Configuración de octavas y notas
current_octave = 4
min_octave = 1
max_octave = 7
num_octavas = 1          # 1 (12 teclas) o 2 (24 teclas)
notes_in_octave = ["C", "Db", "D", "Eb", "E", "F",
                   "Gb", "G", "Ab", "A", "Bb", "B"]
octave_sounds = []

# Diccionario de botones de UI
button_height = 30
button_font_scale = 0.5
button_thickness = 1
buttons = {
    "reset":    {"pos": (10, 10),  "size": (60, button_height),
                 "bg": (50, 50, 50), "text": "Rst"},
    "toggle":   {"pos": (80, 10),  "size": (70, button_height),
                 "bg": (50, 50, 50), "text": "Togl"},
    "frontal":  {"pos": (160, 10), "size": (70, button_height),
                 "bg": (50, 50, 50), "text": "Frnt"},
    "oct_down": {"pos": (240, 10), "size": (60, button_height),
                 "bg": (50, 50, 50), "text": "Oct-"},
    "oct_up":   {"pos": (310, 10), "size": (60, button_height),
                 "bg": (50, 50, 50), "text": "Oct+"},
}
text_color = (255, 255, 255)

# Teclas dinámicas o estáticas
notes_detected = {}
last_triggered = {}

# Controles extra (los dos discos verdes de Oct-/Oct+)
control_positions = {"down": None, "up": None}
control_radii     = {"down": 0,      "up": 0}
last_control      = {"down": 0.0,    "up": 0.0}
CONTROL_COOLDOWN  = 0.5
NOTE_COOLDOWN     = 0.5

# Para “snapshot” en modo estático
static_control_positions = {"down": None, "up": None}
static_control_radii     = {"down": 0,      "up": 0}
static_note_blobs        = []    # lista de (pos, radio)
prev_static_keys         = False

# =============================================================================
# Funciones Auxiliares
# =============================================================================
def draw_button(frame, button_key, active_text=None):
    btn = buttons[button_key]
    x, y = btn["pos"]; w, h = btn["size"]
    text = active_text or btn["text"]
    cv2.rectangle(frame, (x, y), (x + w, y + h), btn["bg"], -1)
    (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX,
                                  button_font_scale, button_thickness)
    tx = x + (w - tw)//2; ty = y + (h + th)//2
    cv2.putText(frame, text, (tx, ty), cv2.FONT_HERSHEY_SIMPLEX,
                button_font_scale, text_color, button_thickness, cv2.LINE_AA)

def order_points(pts):
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]; rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]; rect[3] = pts[np.argmax(diff)]
    return rect

def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

# =============================================================================
# Callback de Mouse
# =============================================================================
def click_event(event, x, y, flags, param):
    global selected_points, static_keys, frontal_view
    global current_octave, num_octavas, prev_static_keys
    if event != cv2.EVENT_LBUTTONDOWN: return

    for k, btn in buttons.items():
        bx, by = btn["pos"]; bw, bh = btn["size"]
        if bx <= x <= bx + bw and by <= y <= by + bh:
            if k == "reset":
                selected_points.clear()
                print("Reset puntos.")
            elif k == "toggle":
                static_keys = not static_keys
                print("Modo estático." if static_keys else "Modo dinámico.")
                # Si salimos de estático, limpiamos el snapshot
                if not static_keys:
                    static_note_blobs.clear()
                    static_control_positions["down"] = static_control_positions["up"] = None
                    static_control_radii["down"] = static_control_radii["up"] = 0
            elif k == "frontal":
                frontal_view = not frontal_view
                print("Vista frontal." if frontal_view else "Normal.")
            elif k == "oct_down":
                if current_octave > min_octave:
                    current_octave -= 1; update_octave_sounds()
                    print("Oct bajada:", current_octave)
                else:
                    print("Oct mínima.")
            elif k == "oct_up":
                if ((num_octavas == 1 and current_octave < max_octave) or
                    (num_octavas == 2 and current_octave < max_octave - 1)):
                    current_octave += 1; update_octave_sounds()
                    print("Oct subida:", current_octave)
                else:
                    print("Oct máxima.")
            prev_static_keys = static_keys
            return

    if len(selected_points) < 4:
        selected_points.append((x, y))
        print("Punto:", (x, y))

# =============================================================================
# Carga y mapeo de sonidos
# =============================================================================
pygame.mixer.init()
def load_sound(path):
    try: return pygame.mixer.Sound(path)
    except Exception as e:
        print(f"Error cargando {path}: {e}")
        return None

def update_octave_sounds():
    global octave_sounds, notes_detected, last_triggered
    temp = []
    octs = [current_octave] if num_octavas == 1 else [current_octave, current_octave+1]
    for o in octs:
        for n in notes_in_octave:
            fn = f"Piano_Samples/{n}{o}.mp3"
            s = load_sound(fn)
            if s is None:
                print("Falta muestra:", fn, "–> cancelado")
                return
            temp.append(s)
    octave_sounds[:] = temp
    print("Sonidos para octavas", octs)
    # Remapeo estático (no borra snapshot)
    if static_keys and notes_detected:
        items = sorted(notes_detected.items(), key=lambda it: it[1]["pos"][0])
        needed = 12 * num_octavas
        if len(items) == needed:
            newd = {}
            for i, (old, d) in enumerate(items):
                oi = (current_octave if num_octavas==1
                      else current_octave + i//12)
                nl = notes_in_octave[i % 12]
                nm = f"{nl}{oi}"
                d["sound"] = octave_sounds[i]
                newd[nm] = d
                last_triggered[nm] = last_triggered.pop(old, 0)
            notes_detected.clear(); notes_detected.update(newd)
        else:
            print("Remapeo estático falló, limpiando notas.")
            notes_detected.clear(); last_triggered.clear()

# Inicializar
update_octave_sounds()

# =============================================================================
# Mediapipe y OpenCV Init
# =============================================================================
mp_hands = mp.solutions.hands
hands    = mp_hands.Hands(static_image_mode=False, max_num_hands=3,
                          min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No hay cámara."); exit()
cv2.namedWindow("Paper Piano", cv2.WINDOW_NORMAL)
cv2.setMouseCallback("Paper Piano", click_event)

# Trackbars HSV
def nothing(x): pass
cv2.namedWindow("HSV Adjust", cv2.WINDOW_AUTOSIZE)
for param, init, m in [("Hmin",50,179),("Smin",100,255),("Vmin",20,255),
                       ("Hmax",100,179),("Smax",255,255),("Vmax",150,255)]:
    cv2.createTrackbar(param, "HSV Adjust", init, m, nothing)

# Presets HSV
HSV_presets = {
    "Verde Oscuro":{"Hmin":85,"Smin":132,"Vmin":113,"Hmax":99,"Smax":255,"Vmax":255},
    "Verde Medio": {"Hmin":35,"Smin":80, "Vmin":50, "Hmax":85,"Smax":255,"Vmax":255},
    "Verde Claro": {"Hmin":50,"Smin":50, "Vmin":80, "Hmax":71,"Smax":82, "Vmax":255},
}
def set_HSV_trackbars(p):
    for k,v in p.items():
        cv2.setTrackbarPos(k, "HSV Adjust", v)
    print("Preset HSV:", p)

# =============================================================================
# Bucle Principal
# =============================================================================
while True:
    ret, frame = cap.read()
    if not ret: break

    # 1) Dibujar UI
    for k in buttons: draw_button(frame, k)
    info_oct = (f"Oct:{current_octave}" if num_octavas==1
                else f"Oct:{current_octave} y {current_octave+1}")
    ox, oy = buttons["oct_up"]["pos"]; ow, _ = buttons["oct_up"]["size"]
    cv2.putText(frame, info_oct, (ox+ow+10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
    for pt in selected_points:
        cv2.circle(frame, pt, 4, (255,0,0), -1)

    # 2) Perspectiva + Efectos
    if len(selected_points) == 4:
        pts  = np.array(selected_points, dtype="float32")
        rect = order_points(pts)
        wA = np.linalg.norm(rect[2]-rect[3]); wB = np.linalg.norm(rect[1]-rect[0])
        hA = np.linalg.norm(rect[1]-rect[2]); hB = np.linalg.norm(rect[0]-rect[3])
        maxW, maxH = int(max(wA,wB)), int(max(hA,hB))
        dst = np.array([[0,0],[maxW-1,0],[maxW-1,maxH-1],[0,maxH-1]], dtype="float32")
        M = cv2.getPerspectiveTransform(rect, dst)
        proc = (cv2.warpPerspective(frame, M, (maxW,maxH))
                if frontal_view else frame.copy())
    else:
        proc = frame.copy()
    if rotate_180: proc = cv2.rotate(proc, cv2.ROTATE_180)
    if mirror_effect: proc = cv2.flip(proc, 1)
    hF, wF = proc.shape[:2]

    # 3) Máscara HSV
    low  = np.array([cv2.getTrackbarPos("Hmin","HSV Adjust"),
                     cv2.getTrackbarPos("Smin","HSV Adjust"),
                     cv2.getTrackbarPos("Vmin","HSV Adjust")], dtype=np.uint8)
    high = np.array([cv2.getTrackbarPos("Hmax","HSV Adjust"),
                     cv2.getTrackbarPos("Smax","HSV Adjust"),
                     cv2.getTrackbarPos("Vmax","HSV Adjust")], dtype=np.uint8)
    hsv  = cv2.cvtColor(proc, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, low, high)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  np.ones((3,3),np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
    cv2.imshow("Mask Green", mask)

    # 4) Detección de círculos verdes
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
    circles = []
    for cnt in contours:
        if cv2.contourArea(cnt) < 50: continue
        (x, y), r = cv2.minEnclosingCircle(cnt)
        circles.append(((int(x), int(y)), int(r)))
    circles.sort(key=lambda t: t[0][0])
    expected = 12 * num_octavas + 2

    # Si acabamos de entrar en estático, tomamos snapshot si hay justo expected blobs
    if static_keys and not prev_static_keys:
        if len(circles) == expected:
            static_control_positions["down"], static_control_radii["down"] = circles[0]
            static_control_positions["up"],   static_control_radii["up"]   = circles[1]
            static_note_blobs[:] = circles[2:]
            print("Snapshot estático guardado.")
        else:
            print(f"No hay {expected} discos para snapshot estático.")

    # Ahora elegimos snapshot vs dinámica
    if static_keys and static_note_blobs:
        # Modo estático: reusar snapshot
        control_positions, control_radii = (static_control_positions,
                                            static_control_radii)
        piano_blobs = static_note_blobs
        if not notes_detected:
            for i, (pos, rad) in enumerate(piano_blobs):
                oi = (current_octave if num_octavas==1
                      else current_octave + i//12)
                nm = f"{notes_in_octave[i%12]}{oi}"
                notes_detected[nm] = {"pos": pos, "radius": rad,
                                      "sound": octave_sounds[i]}
                last_triggered[nm] = 0
    else:
        # Modo dinámico: limpieza de snapshot
        if not static_keys:
            static_note_blobs.clear()
        if len(circles) == expected:
            control_positions["down"], control_radii["down"] = circles[0]
            control_positions["up"],   control_radii["up"]   = circles[1]
            piano_blobs = circles[2:]
            if not static_keys:
                notes_detected.clear(); last_triggered.clear()
                for i, (pos, rad) in enumerate(piano_blobs):
                    oi = (current_octave if num_octavas==1
                          else current_octave + i//12)
                    nm = f"{notes_in_octave[i%12]}{oi}"
                    notes_detected[nm] = {"pos": pos, "radius": rad,
                                          "sound": octave_sounds[i]}
                    last_triggered[nm] = 0
        else:
            if not static_keys:
                control_positions["down"] = control_positions["up"] = None
                control_radii["down"] = control_radii["up"] = 0

    prev_static_keys = static_keys  # actualizar estado

    # 5) Dibujar teclas
    for nm, d in notes_detected.items():
        cv2.circle(proc, d["pos"], d["radius"], (0,255,0), 2)
        cv2.putText(proc, nm,
                    (d["pos"][0]-15, d["pos"][1]+5),
                    cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),2,cv2.LINE_AA)

    # 6) Dibujar controles Oct-/Oct+
    for ctrl, label in (("down","Oct-"),("up","Oct+")):
        p = control_positions.get(ctrl); r = control_radii.get(ctrl,0)
        if p:
            cv2.circle(proc, p, r, (0,0,255), 2)
            cv2.putText(proc, label,
                        (p[0]-r, p[1]-r-5),
                        cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2,cv2.LINE_AA)

    # 7) MediaPipe + colisiones
    rgb = cv2.cvtColor(proc, cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)
    if res.multi_hand_landmarks:
        for hand in res.multi_hand_landmarks:
            mp_drawing.draw_landmarks(proc, hand, mp_hands.HAND_CONNECTIONS)
            for idx in [4,8,12,16,20]:
                lm = hand.landmark[idx]
                px,py = int(lm.x*wF), int(lm.y*hF)
                cv2.circle(proc,(px,py),4,(0,255,255),-1)
                now = time.time()
                # controles
                for ctrl in ("down","up"):
                    cp = control_positions[ctrl]
                    if cp and distance((px,py),cp) < control_radii[ctrl] \
                       and now - last_control[ctrl] > CONTROL_COOLDOWN:
                        if ctrl=="down" and current_octave>min_octave:
                            current_octave -=1; update_octave_sounds()
                        if ctrl=="up" and ((num_octavas==1 and current_octave<max_octave) or
                                           (num_octavas==2 and current_octave<max_octave-1)):
                            current_octave +=1; update_octave_sounds()
                        last_control[ctrl] = now
                # notas
                for nm, d in notes_detected.items():
                    if distance((px,py), d["pos"]) < d["radius"] \
                       and now - last_triggered.get(nm,0) > NOTE_COOLDOWN:
                        d["sound"].play(); last_triggered[nm] = now

    # 8) Mostrar y manejar teclado físico
    cv2.imshow("Paper Piano", proc)
    key = cv2.waitKey(1) & 0xFF
    if key in [ord('r'),ord('t'),ord('f'),ord('g'),ord('m'),
               ord('p'),ord('o'),ord('n'),ord('h'),ord('j'),ord('k'),ord('q')]:
        if key==ord('r'): selected_points.clear()
        elif key==ord('t'): static_keys = not static_keys
        elif key==ord('f'): frontal_view = not frontal_view
        elif key==ord('g'): rotate_180 = not rotate_180
        elif key==ord('m'): mirror_effect = not mirror_effect
        elif key==ord('p') and ((num_octavas==1 and current_octave<max_octave) or
                                (num_octavas==2 and current_octave<max_octave-1)):
            current_octave +=1; update_octave_sounds()
        elif key==ord('o') and current_octave>min_octave:
            current_octave -=1; update_octave_sounds()
        elif key==ord('n'):
            num_octavas = 2 if num_octavas==1 else 1
            update_octave_sounds()
        elif key==ord('h'): set_HSV_trackbars(HSV_presets["Verde Oscuro"])
        elif key==ord('j'): set_HSV_trackbars(HSV_presets["Verde Medio"])
        elif key==ord('k'): set_HSV_trackbars(HSV_presets["Verde Claro"])
        elif key==ord('q'): break

cap.release()
cv2.destroyAllWindows()


Sonidos para octavas [4]
Punto: (2, 469)
Punto: (84, 182)
Punto: (636, 466)
Punto: (554, 192)
Preset HSV: {'Hmin': 85, 'Smin': 132, 'Vmin': 113, 'Hmax': 99, 'Smax': 255, 'Vmax': 255}
Preset HSV: {'Hmin': 35, 'Smin': 80, 'Vmin': 50, 'Hmax': 85, 'Smax': 255, 'Vmax': 255}
Preset HSV: {'Hmin': 85, 'Smin': 132, 'Vmin': 113, 'Hmax': 99, 'Smax': 255, 'Vmax': 255}
Preset HSV: {'Hmin': 50, 'Smin': 50, 'Vmin': 80, 'Hmax': 71, 'Smax': 82, 'Vmax': 255}
Preset HSV: {'Hmin': 85, 'Smin': 132, 'Vmin': 113, 'Hmax': 99, 'Smax': 255, 'Vmax': 255}
Snapshot estático guardado.
Snapshot estático guardado.
Sonidos para octavas [5]
Sonidos para octavas [6]
Sonidos para octavas [7]
Sonidos para octavas [6]
Sonidos para octavas [5]
Sonidos para octavas [6]
Sonidos para octavas [5]
Sonidos para octavas [6]
Sonidos para octavas [5]
Sonidos para octavas [6]
Sonidos para octavas [5]
Sonidos para octavas [6]
Sonidos para octavas [5]
Sonidos para octavas [6]
Sonidos para octavas [5]
Sonidos para octavas [6]
Sonidos pa

# Referencias

## Similar Projects
- BTifmmp. (2024). GitHub - BTifmmp/paper-piano: Paper Piano uses Python and OpenCV to detect key presses on a hand-drawn piano, translating them into digital notes and sound. GitHub. https://github.com/BTifmmp/paper-piano
- google-ai-edge. (2025, February 7). GitHub - google-ai-edge/mediapipe: Cross-platform, customizable ML solutions for live and streaming media. GitHub. https://github.com/google-ai-edge/mediapipe
- Reddit - Explora lo que quieras. (2025). Reddit.com. https://www.reddit.com/r/computervision/comments/1es14rt/i_made_piano_on_paper_using_python_opencv_and/?rdt=62406
- MustafaLotfi. (2022). GitHub - MustafaLotfi/Paper-Piano: Using this program, you can play or control music by hand gesture. Also you can play a Paper Piano. GitHub. https://github.com/MustafaLotfi/Paper-Piano
- Saltwash. (2015, October 22). Paper Piano using Python and OpenCV. Electric Soup. https://rdmilligan.wordpress.com/2015/10/22/paper-piano-using-python-and-opencv/
- Instructables. (2021, December 12). Webcam Motion Piano With Python and OpenCV. Instructables. https://www.instructables.com/Webcam-Motion-Piano-With-Python-and-OpenCV2/
- (2025). Youtube.com. https://www.youtube.com/watch?v=F13tCcMjRLk&ab_channel=UmairSheikh

## Instrument Samples
- fuhton. (2017). GitHub - fuhton/piano-mp3: MP3 exports of piano keys. GitHub. https://github.com/fuhton/piano-mp3/tree/master
- nbrosowsky. (2018). GitHub - nbrosowsky/tonejs-instruments: A small instrument sample library with quick-loader for tone.js. GitHub. https://github.com/nbrosowsky/tonejs-instruments
- Freesound. (2025). Freesound.org. https://freesound.org/
- Free samples from Karoryfer. (2025). Karoryfer Samples. https://shop.karoryfer.com/pages/free-samples
- VSCO 2: Community Edition (CE) - Versilian Studios. (2024, September 30). Versilian Studios. https://versilian-studios.com/vsco-community/
- University of Iowa Electronic Music Studios. (2016). Uiowa.edu. https://theremin.music.uiowa.edu/

## Music Theory
- The ABC’s of the Musical Alphabet | Hub Guitar. (2018). Hubguitar.com. https://hubguitar.com/music-theory/the-musical-alphabet#:~:text=Memorize%20the%207%20letters%20of,A%E2%99%AF%2C%20B%2C%20C.